<a href="https://colab.research.google.com/github/Maironsantana/TCC-PDF-keywords-Temas-da-computacao/blob/main/TCC_Sistema_Recomendativo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Bibliotecas

In [2]:
!pip install unicode
import unicodedata
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


#Carregando Dataframe

In [4]:
#Fazendo upload do arquivo csv com a seguinte estrutura:
#Orientador / Coorientador / "Tema_1" / "Tema_2" / "Tema_3" / "Tema_4" / ... (Substituindo os "temas" pelo nome do tema propriamente dito)
#Onde cada linha do csv é um trabalho com seus orientadores, coorientadores e valores de porcentagem em cada tema (Usar o script para classificar trabalhos)

TCCsDataset = pd.read_csv('/content/drive/MyDrive/OrientadoresTCCs.csv')

In [ ]:
#Preenchendo trabalhos sem coorientadores com "-"
TCCsDataset = TCCsDataset.fillna("-")
TCCsDataset

A seguir, iremos buscar os 3 temas com maiores valores de porcentagem, assim cada trabalho terá 3 temas relacionados

In [ ]:
# Lista para armazenar os novos dados
novo_array_de_trabalhos = []

# Itera sobre as linhas do DataFrame
for _, linha in TCCsDataset.iterrows():
    # Extrai os nomes dos orientadores e coorientadores
    orientador = linha['Orientador']
    coorientador = linha['Coorientador']

    # Cria um dicionário para armazenar as pontuações dos temas
    temas_pontuacoes = {tema: linha[tema] for tema in TCCsDataset.columns[2:]}

    # Obtém os cinco temas de maior pontuação
    temas_mais_pontuados = sorted(temas_pontuacoes, key=temas_pontuacoes.get, reverse=True)[:5]

    # Adiciona os dados ao novo DataFrame
    novo_array_de_trabalhos.append({
        'Orientador': orientador,
        'Coorientador': coorientador,
        'Tema1': temas_mais_pontuados[0],
        'Tema2': temas_mais_pontuados[1],
        'Tema3': temas_mais_pontuados[2],
    })

# Cria um novo DataFrame com os dados gerados
novo_TCCsDataset = pd.DataFrame(novo_array_de_trabalhos)

# Exibe o novo DataFrame
novo_TCCsDataset


,Orientador,Coorientador,Tema1,Tema2,Tema3
0,FELIPE BASTOS NUNES,MARIO WEDNEY DE LIMA MOREIRA,desenvolvimento_web,linguagens_de_programacao_de_alto_nivel,desenvolvimento_de_software
1,RICARDO LENZ CESAR,-,desenvolvimento_web,linguagens_de_programacao_de_alto_nivel,protocolos_de_comunicacao
2,HENRIQUE VIANA OLIVEIRA,-,padroes_e_boas_praticas_de_codificacao,desenvolvimento_web,automatos_finitos_deterministicos
3,MARIO WEDNEY DE LIMA MOREIRA,-,arquitetura_de_redes,redes_neurais_artificiais,processamento_de_imagens
4,THIAGO FELIPPE DE LIMA BANDEIRA,DIEGO ROCHA LIMA,sistemas_distribuidos,aprendizado_de_maquina,complexidade_computacional
...,...,...,...,...,...
77,FRANCISCA RAQUEL DE VASCONCELOS SILVEIRA,-,desenvolvimento_web,modelo_relacional,sql
78,FRANCISCA RAQUEL DE VASCONCELOS SILVEIRA,-,comunicacao_sem_fio,complexidade_computacional,logica_de_primeira_ordem
79,FRANCISCA RAQUEL DE VASCONCELOS SILVEIRA,-,gestao_de_qualidade_de_software,metodologias_ageis,desenvolvimento_de_software
80,REINALDO BEZERRA BRAGA,-,linguagens_de_programacao_de_alto_nivel,padroes_e_boas_praticas_de_codificacao,desenvolvimento_web


Para descobrir o tempo de experiência de um professor em um determinado tema, e feito uma relação entre Tema - Subárea da Computação - Disciplina. Sabendo o tempo de experiencia de um professor em uma determinada disciplina é possivel descobrir o mesmo para um tema da computação.

In [ ]:
#Importação de um CSV fazendo a relação de uma matéra com a subárea da computação
relacaoDisciplinaSubarea = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/datasets/relacao_materia-subarea.csv', index_col='MATÉRIA')
relacaoDisciplinaSubarea

,SUBÁREA
MATÉRIA,
INTRODUÇÃO À COMPUTAÇÃO,-
CÁLCULO DIFERENCIAL E INTEGRAL I,-
INTRODUÇÃO À PROGRAMAÇÃO,LÓGICA E SEMÂNTICA DE PROGRAMAS
PORTUGUÊS INSTRUMENTAL,-
MATEMÁTICA DISCRETA,MATEMÁTICA DA COMPUTAÇÃO
ARQUITETURA E ORGANIZAÇÃO DE COMPUTADORES,SISTEMAS DE COMPUTAÇÃO
CÁLCULO DIFERENCIAL E INTEGRAL II,-
ESTRUTURA DE DADOS,LÓGICA E SEMÂNTICA DE PROGRAMAS
FÍSICA APLICADA,-


In [ ]:
#Importação de um CSV fazendo a relação de um tema com a subárea da computação
relacaoTopicoSubarea = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/datasets/relacao-topico-subarea.csv', index_col='TÓPICO')
relacaoTopicoSubarea

In [ ]:
#Importando um CSV contendo a relação professor X MATÉRIA, onde cada célula possui o valor em horas do professor na MATÉRIA
DatasetHorasDisciplinas = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/datasets/DatasetHorasDisciplinas.csv', index_col='NOME_PROFESSOR')
DatasetHorasDisciplinas

In [ ]:
#Algoritmo para calcular o tempo de experiência em horas de um professor na SUBÁREA da computação
DicionarioHorasSubareas = {}

for index, row in DatasetHorasDisciplinas.iterrows():
  for column, value in row.items():
    if index not in DicionarioHorasSubareas:
      DicionarioHorasSubareas[index] = {}
    if relacaoDisciplinaSubarea.loc[column, 'SUBÁREA'] in DicionarioHorasSubareas[index]:
      DicionarioHorasSubareas[index][relacaoDisciplinaSubarea.loc[column, 'SUBÁREA']] += value
    else:
      DicionarioHorasSubareas[index][relacaoDisciplinaSubarea.loc[column, 'SUBÁREA']] = value


#Temos agora um dataset com a relação professor X SUBÁREA
DatasetHorasSubareas = pd.DataFrame(DicionarioHorasSubareas)
DatasetHorasSubareas = DatasetHorasSubareas.transpose()
#DROPANDO MATERIAS SEM CLASSIFICAÇÃO
DatasetHorasSubareas = DatasetHorasSubareas.drop('-', axis=1)
DatasetHorasSubareas

In [ ]:
#Algoritmo para calcular o tempo de experiência em horas de um professor no TEMA da computação
DicionarioHorasTopicos = {}

for index, row in relacaoTopicoSubarea.iterrows():
  for professor, subarea in DatasetHorasSubareas.iterrows():
    if professor not in DicionarioHorasTopicos:
      DicionarioHorasTopicos[professor] = {}

    if row['SUBÁREA'] not in DatasetHorasSubareas.columns:
      DicionarioHorasTopicos[professor][index] = 0
    else:
      DicionarioHorasTopicos[professor][index] = subarea[row['SUBÁREA']]

#Temos agora um dataset com a relação professor X TEMA
DatasetHorasTopicos = pd.DataFrame(DicionarioHorasTopicos)
DatasetHorasTopicos = DatasetHorasTopicos.transpose()
DatasetHorasTopicos

#Análise dos dados

In [ ]:
# Concatenar as colunas de temas em uma única série
temas_series = pd.concat([novo_TCCsDataset['Tema1'], novo_TCCsDataset['Tema2'], novo_TCCsDataset['Tema3']])

# Calcular a contagem total de aparições para cada tema
contagem_temas = temas_series.value_counts()

# Calcular a porcentagem de aparição de cada tema
porcentagem_temas = (contagem_temas / 82)

# Ordenar os temas por porcentagem
porcentagem_temas = porcentagem_temas.sort_values(ascending=False)
df_maiores_valores = porcentagem_temas.head(10)

# Criar um gráfico de barras
plt.figure(figsize=(10, 6))
df_maiores_valores.plot(kind='bar')
plt.xlabel('TÓPICOS')
plt.ylabel('PORCENTAGEM DE TRABALHOS')
plt.show()


In [ ]:
#PARTICIPAÇÃO DOS PROFESSORES NOS TCCS APRESENTADOS
orientadores_unicos = TCCsDataset['Orientador'].append(TCCsDataset['Coorientador']).unique()

df_porcentagens_orientadores = pd.DataFrame(columns=['Porcentagem de Aparição'])

for orientador in orientadores_unicos:
    total_aparicoes = ((TCCsDataset['Orientador'] == orientador) | (TCCsDataset['Coorientador'] == orientador)).sum()

    porcentagem = total_aparicoes / len(TCCsDataset)

    df_porcentagens_orientadores.loc[orientador] = [porcentagem]

df_porcentagens_orientadores

<ipython-input-14-a184ef2565c1>:2: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  orientadores_unicos = TCCsDataset['Orientador'].append(TCCsDataset['Coorientador']).unique()


,Porcentagem de Aparição
FELIPE BASTOS NUNES,0.085366
RICARDO LENZ CESAR,0.036585
HENRIQUE VIANA OLIVEIRA,0.073171
MARIO WEDNEY DE LIMA MOREIRA,0.121951
THIAGO FELIPPE DE LIMA BANDEIRA,0.048780
KILBERT AMORIM MACIEL,0.012195
ANDRESSA BEZERRA FERREIRA,0.012195
REINALDO BEZERRA BRAGA,0.195122
DIEGO ROCHA LIMA,0.146341
ALEXANDRO LIMA DAMASCENO,0.036585


#Pontuação individual

In [ ]:
#é feita uma soma para descobrir o valor total de participação de cada professor nos temas
PontuacaoTCC = TCCsDataset
dataset_columns = PontuacaoTCC.columns[2:]
dataset_columns

total_professores = {}

for index, row in PontuacaoTCC.iterrows():
  orientador = row['Orientador']
  coorientador = row['Coorientador']


  if orientador not in total_professores:
    total_professores[orientador] = {}
    total_professores[orientador] = row[dataset_columns]
  elif orientador in total_professores:
    total_professores[orientador] += row[dataset_columns]

  if coorientador not in total_professores:
    total_professores[coorientador] = {}
    total_professores[coorientador] = row[dataset_columns]
  elif coorientador in total_professores:
    total_professores[coorientador] += row[dataset_columns]


total_orientadores_df = pd.DataFrame(total_professores)
total_orientadores_df = total_orientadores_df.drop(columns = '-', axis=1)

total_orientadores_df

,FELIPE BASTOS NUNES,MARIO WEDNEY DE LIMA MOREIRA,RICARDO LENZ CESAR,HENRIQUE VIANA OLIVEIRA,THIAGO FELIPPE DE LIMA BANDEIRA,DIEGO ROCHA LIMA,KILBERT AMORIM MACIEL,ANDRESSA BEZERRA FERREIRA,REINALDO BEZERRA BRAGA,ALEXANDRO LIMA DAMASCENO,CARINA TEIXEIRA DE OLIVEIRA,RAIMUNDO VALTER COSTA FILHO,SILAS SANTIAGO LOPES PEREIRA,ANTONIO MAURO BARBOSA DE OLIVEIRA,FRANCISCA RAQUEL DE VASCONCELOS SILVEIRA,ERICA DE LIMA GALLINDO,JOSE WILLIAM ALVES DA SILVA
complexidade_computacional,0.020836,0.072891,0.0,0.163043,0.144886,0.82615,0.0,0.0,0.103938,0.010526,0.187435,0.0,0.024716,0.039326,0.178037,0.0,0.0
teoria_da_recursao,0.0,0.0,0.0,0.0,0.0,0.001753,0.0,0.0,0.0,0.0,0.032616,0.0,0.0,0.003096,0.0,0.0,0.0
teoria_dos_grafos,0.010526,0.022472,0.020737,0.344532,0.014583,0.303767,0.013333,0.0,0.033789,0.013442,0.120238,0.058824,0.038973,0.045494,0.058436,0.0,0.0
computacao_quantica,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
maquina_de_turing,0.0,0.007752,0.004494,0.0,0.0,0.0,0.0,0.0,0.011494,0.0,0.01845,0.0,0.0,0.0,0.008547,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
redes_neurais_artificiais,0.104056,1.449849,0.006897,0.055314,0.059172,0.0429,0.0,0.010753,0.649309,0.029155,0.60132,0.296296,1.348795,0.286215,0.093317,0.0,0.016667
sistemas_especialistas,0.010526,0.128647,0.016242,0.043135,0.022727,0.152089,0.026667,0.0,0.112472,0.010526,0.200235,0.041757,0.147372,0.420397,0.264731,0.0,0.004167
processamento_de_linguagem_natural,0.010526,0.409029,0.028037,0.511657,0.024242,0.148288,0.0,0.0,0.171466,0.010526,0.176048,0.205882,0.137584,0.010638,0.399849,0.176471,0.0
visao_computacional,0.0,0.0625,0.0,0.007576,0.005682,0.01632,0.0,0.0,0.049469,0.0,0.045355,0.061728,0.109674,0.0,0.0,0.0,0.0


In [ ]:
#Montagem da tabela de probabilidade. Cada professor terá um valor de porcentagem calculado a partir do valor de trabalhos no tema que ele orientou, dividido pelo total de trabalhos no tema

PontuacaoTCC = total_orientadores_df

def calcular_porcentagem(linha):
    soma = linha.sum()
    if soma == 0:
        return linha  # Evita a divisão por zero, mantendo os valores originais
    return (linha / soma)

dataset_columns = PontuacaoTCC.apply(calcular_porcentagem, axis=1)

dataset_columns

,FELIPE BASTOS NUNES,MARIO WEDNEY DE LIMA MOREIRA,RICARDO LENZ CESAR,HENRIQUE VIANA OLIVEIRA,THIAGO FELIPPE DE LIMA BANDEIRA,DIEGO ROCHA LIMA,KILBERT AMORIM MACIEL,ANDRESSA BEZERRA FERREIRA,REINALDO BEZERRA BRAGA,ALEXANDRO LIMA DAMASCENO,CARINA TEIXEIRA DE OLIVEIRA,RAIMUNDO VALTER COSTA FILHO,SILAS SANTIAGO LOPES PEREIRA,ANTONIO MAURO BARBOSA DE OLIVEIRA,FRANCISCA RAQUEL DE VASCONCELOS SILVEIRA,ERICA DE LIMA GALLINDO,JOSE WILLIAM ALVES DA SILVA
complexidade_computacional,0.011760,0.041140,0.000000,0.092022,0.081774,0.466281,0.000000,0.000000,0.058663,0.005941,0.105789,0.000000,0.013950,0.022196,0.100484,0.000000,0.000000
teoria_da_recursao,0.000000,0.000000,0.000000,0.000000,0.000000,0.046786,0.000000,0.000000,0.000000,0.000000,0.870578,0.000000,0.000000,0.082636,0.000000,0.000000,0.000000
teoria_dos_grafos,0.009577,0.020445,0.018866,0.313454,0.013268,0.276366,0.012131,0.000000,0.030741,0.012229,0.109392,0.053517,0.035458,0.041390,0.053165,0.000000,0.000000
computacao_quantica,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
maquina_de_turing,0.000000,0.152784,0.088581,0.000000,0.000000,0.000000,0.000000,0.000000,0.226542,0.000000,0.363638,0.000000,0.000000,0.000000,0.168455,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
redes_neurais_artificiais,0.020605,0.287098,0.001366,0.010953,0.011717,0.008495,0.000000,0.002129,0.128576,0.005773,0.119073,0.058672,0.267087,0.056676,0.018479,0.000000,0.003300
sistemas_especialistas,0.006572,0.080320,0.010141,0.026931,0.014190,0.094956,0.016649,0.000000,0.070221,0.006572,0.125015,0.026071,0.092010,0.262471,0.165282,0.000000,0.002601
processamento_de_linguagem_natural,0.004349,0.169003,0.011585,0.211407,0.010017,0.061270,0.000000,0.000000,0.070846,0.004349,0.072740,0.085067,0.056847,0.004396,0.165210,0.072914,0.000000
visao_computacional,0.000000,0.174433,0.000000,0.021143,0.015858,0.045548,0.000000,0.000000,0.138064,0.000000,0.126582,0.172280,0.306091,0.000000,0.000000,0.000000,0.000000


In [ ]:
#Adição de valores ficticios para evitar futuramente multiplicações por 0 no algoritmo
TCCDataframe = dataset_columns.transpose()
TCCDataframe = TCCDataframe + 0.001
TCCDataframe

In [ ]:
#adição de valores ficticios para evitar multiplicações por 0 no algoritmo
DatasetHorasTopicos = DatasetHorasTopicos + 1
DatasetHorasTopicos

In [5]:
#Calculo do valor de probabilidade do professor relacionado a quantidade de horas que ele ensinou na matéria/tema
TopicosHorasDataframe = DatasetHorasTopicos.div(DatasetHorasTopicos.sum(axis=0).replace(0,1), axis=1)

def formatar_nome_coluna(nome):
    nome = nome.lower()  # Converte para minúsculas
    nome = nome.replace(" ", "_")  # Substitui espaços por underscores
    nome = ''.join((c for c in unicodedata.normalize('NFD', nome) if unicodedata.category(c) != 'Mn'))  # Remove acentos
    return nome

TopicosHorasDataframe.columns = [formatar_nome_coluna(coluna) for coluna in TopicosHorasDataframe.columns]
TopicosHorasDataframe

NameError: name 'DatasetHorasTopicos' is not defined

In [ ]:
#É feito o merge de ambos os datasets Orientação e Horas ministradas

professores = list(set(TCCDataframe.index))
topicos = list(set(TCCDataframe.columns))

colunas = pd.MultiIndex.from_product([topicos, ['Participação', 'Horas Ministradas']])

TeachersCrossMerged = pd.DataFrame(1, index=professores, columns=colunas)

for professor in professores:
  for topico in topicos:
    if topico in TCCDataframe.columns and professor in TCCDataframe.index:
      tcc_value = TCCDataframe.at[professor, topico]
      TeachersCrossMerged.at[professor, (topico, 'Participação')] = tcc_value
    if topico in TopicosHorasDataframe.columns and professor in TopicosHorasDataframe.index:
      hours_value = TopicosHorasDataframe.at[professor, topico]
      TeachersCrossMerged.at[professor, (topico, 'Horas Ministradas')] = hours_value

TeachersCrossMerged

#Algorítimo

Primeiro modelo do algoritmo, com testes mockados

In [ ]:

#GERAR A RECOMENDAÇÃO
def gerar_recomendação(assunto1, assunto2, assunto3):
  recomendacao = {}
  for index, row in TeachersCrossMerged.iterrows():
    P_Assunto1 = TeachersCrossMerged.loc[index, (assunto1, 'Participação')]
    H_Assunto1 = TeachersCrossMerged.loc[index, (assunto1, 'Horas Ministradas')]

    P_Assunto2 = TeachersCrossMerged.loc[index, (assunto2, 'Participação')]
    H_Assunto2 = TeachersCrossMerged.loc[index, (assunto2, 'Horas Ministradas')]

    P_Assunto3 = TeachersCrossMerged.loc[index, (assunto3, 'Participação')]
    H_Assunto3 = TeachersCrossMerged.loc[index, (assunto3, 'Horas Ministradas')]


    P_total = df_porcentagens_orientadores.at[index, 'Porcentagem de Aparição']

    P = P_total * P_Assunto1 * P_Assunto2 * P_Assunto3 * H_Assunto1 * H_Assunto2 * H_Assunto3
    recomendacao[index] = P


  return recomendacao

#RESULTADO DA PREVISÃO
previsao = gerar_recomendação('aprendizado_de_maquina', 'aprendizado_de_maquina', 'aprendizado_de_maquina')


#CALCULO DA PORCENTAGEM
soma_total = sum(previsao.values())

porcentagens = {}
for autor, resultado in previsao.items():
    porcentagem = (resultado / soma_total) * 100
    porcentagens[autor] = porcentagem

porcentagens
# previsao


{'FRANCISCA RAQUEL DE VASCONCELOS SILVEIRA': 0.3785732604896552,
 'RICARDO LENZ CESAR': 2.9303150242803827e-12,
 'KILBERT AMORIM MACIEL': 8.182966133276733e-15,
 'SILAS SANTIAGO LOPES PEREIRA': 98.36959922383276,
 'HENRIQUE VIANA OLIVEIRA': 6.698484354805232e-10,
 'DIEGO ROCHA LIMA': 4.116931030287112e-08,
 'CARINA TEIXEIRA DE OLIVEIRA': 0.914013153690674,
 'ANTONIO MAURO BARBOSA DE OLIVEIRA': 2.688599011667018e-10,
 'MARIO WEDNEY DE LIMA MOREIRA': 1.5896736325520331e-07,
 'JOSE WILLIAM ALVES DA SILVA': 8.182966133276733e-15,
 'ALEXANDRO LIMA DAMASCENO': 2.7005251997767e-13,
 'ANDRESSA BEZERRA FERREIRA': 8.182966133276733e-15,
 'RAIMUNDO VALTER COSTA FILHO': 0.33781408368543553,
 'FELIPE BASTOS NUNES': 4.093105320483921e-11,
 'THIAGO FELIPPE DE LIMA BANDEIRA': 2.248701186521249e-09,
 'ERICA DE LIMA GALLINDO': 1.902157026090814e-09,
 'REINALDO BEZERRA BRAGA': 7.303111748219704e-08}

In [ ]:
def algoritmo_recomendacao(assunto1, assunto2, assunto3):
  previsao = gerar_recomendação(assunto1, assunto2, assunto3)
  series = pd.Series(previsao)
  qtd = 5
  melhores_professores = series.nlargest(5)
  return melhores_professores

algoritmo_recomendacao('complexidade_computacional', 'complexidade_computacional', 'complexidade_computacional')

DIEGO ROCHA LIMA                            8.665746e-04
HENRIQUE VIANA OLIVEIRA                     1.014072e-06
CARINA TEIXEIRA DE OLIVEIRA                 8.458341e-15
FRANCISCA RAQUEL DE VASCONCELOS SILVEIRA    2.989218e-15
REINALDO BEZERRA BRAGA                      1.388348e-15
dtype: float64

#Validação

Testes de validação para calcular a acurácia simples

In [ ]:
#SEPARAÇÃO DOS 13 ARTIGOS PARA TESTE

contagem_orientadores = TCCsDataset['Orientador'].value_counts()

total_test_items = 13
orientador_proportions = contagem_orientadores / contagem_orientadores.sum()
artigos_por_orientador = (orientador_proportions * 20).round().astype(int)

TestTCCsDataset = pd.DataFrame()
for orientador, num_artigos in artigos_por_orientador.items():
    orientador_data = TCCsDataset[TCCsDataset['Orientador'] == orientador].sample(n=num_artigos, random_state=13)
    TestTCCsDataset = pd.concat([TestTCCsDataset, orientador_data])


#PREPARAÇÃO DO DATAFRAME DE TREINO
indices_itens_teste = TCCsDataset.index.isin(TestTCCsDataset.index)
TrainTCCsDataset = TCCsDataset[~indices_itens_teste]

#NOVO DATAFRAME DE TESTE: TestTCCsDataset
#NOVO DATAFRAME DE TREINO:
TrainTCCsDataset

A seguir, as mesmas etapas feitas anteriormente, entretando com apenas o dataframe de treino

In [ ]:
PontuacaoTrainTCC = TrainTCCsDataset
dataset_columns = PontuacaoTrainTCC.columns[2:]
dataset_columns

total_professores = {}

for index, row in PontuacaoTrainTCC.iterrows():
  orientador = row['Orientador']
  coorientador = row['Coorientador']


  if orientador not in total_professores:
    total_professores[orientador] = {}
    total_professores[orientador] = row[dataset_columns]
  elif orientador in total_professores:
    total_professores[orientador] += row[dataset_columns]

  if coorientador not in total_professores:
    total_professores[coorientador] = {}
    total_professores[coorientador] = row[dataset_columns]
  elif coorientador in total_professores:
    total_professores[coorientador] += row[dataset_columns]


train_orientadores_df = pd.DataFrame(total_professores)
train_orientadores_df = train_orientadores_df.drop(columns = '-', axis=1)

train_orientadores_df

,FELIPE BASTOS NUNES,MARIO WEDNEY DE LIMA MOREIRA,RICARDO LENZ CESAR,HENRIQUE VIANA OLIVEIRA,THIAGO FELIPPE DE LIMA BANDEIRA,DIEGO ROCHA LIMA,KILBERT AMORIM MACIEL,ANDRESSA BEZERRA FERREIRA,REINALDO BEZERRA BRAGA,ALEXANDRO LIMA DAMASCENO,CARINA TEIXEIRA DE OLIVEIRA,RAIMUNDO VALTER COSTA FILHO,SILAS SANTIAGO LOPES PEREIRA,ANTONIO MAURO BARBOSA DE OLIVEIRA,FRANCISCA RAQUEL DE VASCONCELOS SILVEIRA,ERICA DE LIMA GALLINDO,JOSE WILLIAM ALVES DA SILVA
complexidade_computacional,0.020836,0.072891,0.0,0.0,0.144886,0.254583,0.0,0.0,0.098562,0.010526,0.157914,0.0,0.021361,0.039326,0.176068,0.0,0.0
teoria_da_recursao,0.0,0.0,0.0,0.0,0.0,0.001753,0.0,0.0,0.0,0.0,0.003096,0.0,0.0,0.003096,0.0,0.0,0.0
teoria_dos_grafos,0.010526,0.022472,0.016242,0.007576,0.008523,0.293129,0.013333,0.0,0.026384,0.013442,0.098098,0.058824,0.008772,0.045494,0.034814,0.0,0.0
computacao_quantica,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
maquina_de_turing,0.0,0.007752,0.0,0.0,0.0,0.0,0.0,0.0,0.011494,0.0,0.0,0.0,0.0,0.0,0.008547,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
redes_neurais_artificiais,0.104056,1.137349,0.006897,0.055314,0.056142,0.020678,0.0,0.010753,0.613465,0.029155,0.56837,0.296296,1.293016,0.207268,0.085443,0.0,0.016667
sistemas_especialistas,0.010526,0.128647,0.016242,0.043135,0.022727,0.141451,0.026667,0.0,0.099541,0.010526,0.087673,0.041757,0.110459,0.420397,0.260794,0.0,0.004167
processamento_de_linguagem_natural,0.010526,0.409029,0.028037,0.511657,0.022727,0.148288,0.0,0.0,0.16995,0.010526,0.176048,0.205882,0.0,0.010638,0.356542,0.176471,0.0
visao_computacional,0.0,0.0625,0.0,0.007576,0.005682,0.005682,0.0,0.0,0.049469,0.0,0.037975,0.061728,0.109674,0.0,0.0,0.0,0.0


In [ ]:
PontuacaoTrainTCC = train_orientadores_df

def calcular_porcentagem(linha):
    soma = linha.sum()
    if soma == 0:
        return linha
    return (linha / soma)

dataset_columns = PontuacaoTrainTCC.apply(calcular_porcentagem, axis=1)


TrainDataframe = dataset_columns.transpose()
TrainDataframe = TrainDataframe + 0.001
TrainDataframe

,complexidade_computacional,teoria_da_recursao,teoria_dos_grafos,computacao_quantica,maquina_de_turing,linguagens_recursivas_e_recursivamente_enumeraveis,teoria_da_lambda_calculabilidade,teoria_da_redutibilidade,automatos_finitos_deterministicos,automatos_finitos_nao_deterministicos,...,seguranca_em_redes_de_computadores,mineracao_de_dados,probabilidade_e_estatistica,aprendizado_de_maquina,aprendizado_profundo,redes_neurais_artificiais,sistemas_especialistas,processamento_de_linguagem_natural,visao_computacional,algoritmos_geneticos
FELIPE BASTOS NUNES,0.021899,0.001000,0.017006,0.001,0.001000,0.001,0.001,0.001000,0.028984,0.028984,...,0.001000,0.025349,0.060177,0.009418,0.002238,0.024119,0.008388,0.005707,0.001000,0.024563
MARIO WEDNEY DE LIMA MOREIRA,0.074114,0.001000,0.035171,0.001,0.279915,0.001,0.001,0.111158,0.062747,0.062747,...,0.001000,0.132704,0.083987,0.114412,0.165802,0.253696,0.091297,0.183902,0.184670,0.001000
RICARDO LENZ CESAR,0.001000,0.001000,0.025698,0.001,0.001000,0.001,0.001,0.001000,0.008192,0.008192,...,0.001000,0.001000,0.016374,0.005160,0.005895,0.002532,0.012400,0.013537,0.001000,0.001000
HENRIQUE VIANA OLIVEIRA,0.001000,0.001000,0.012520,0.001,0.001000,0.001,0.001,0.050516,0.211028,0.211028,...,0.001000,0.059671,0.007231,0.025273,0.021145,0.013290,0.031277,0.229794,0.023263,0.001000
THIAGO FELIPPE DE LIMA BANDEIRA,0.146329,0.001000,0.013960,0.001,0.001000,0.001,0.501,0.001000,0.037647,0.037647,...,0.001000,0.004711,0.022920,0.043361,0.042659,0.013474,0.016952,0.011163,0.017697,0.051876
DIEGO ROCHA LIMA,0.256361,0.221628,0.446736,0.001,0.001000,1.001,0.501,0.053223,0.091122,0.091122,...,0.001000,0.035144,0.072299,0.079289,0.061447,0.005594,0.100284,0.067309,0.017697,0.217125
KILBERT AMORIM MACIEL,0.001000,0.001000,0.021275,0.001,0.001000,0.001,0.001,0.175295,0.001000,0.001000,...,0.001000,0.044542,0.022934,0.001000,0.001000,0.001000,0.019717,0.001000,0.001000,0.001000
ANDRESSA BEZERRA FERREIRA,0.001000,0.001000,0.001000,0.001,0.001000,0.001,0.001,0.001000,0.009275,0.009275,...,0.001000,0.008023,0.001000,0.001000,0.006631,0.003389,0.001000,0.001000,0.001000,0.001000
REINALDO BEZERRA BRAGA,0.099863,0.001000,0.041120,0.001,0.414564,0.001,0.001,0.001000,0.078781,0.078781,...,0.269769,0.109428,0.054572,0.086913,0.108055,0.137299,0.070867,0.076995,0.146375,0.009676
ALEXANDRO LIMA DAMASCENO,0.011558,0.001000,0.021440,0.001,0.001000,0.001,0.001,0.001000,0.001000,0.001000,...,0.001000,0.002904,0.001000,0.002298,0.001000,0.007478,0.008388,0.005707,0.001000,0.024563


In [ ]:
professores = list(set(TrainDataframe.index))
topicos = list(set(TrainDataframe.columns))

colunas = pd.MultiIndex.from_product([topicos, ['Participação', 'Horas Ministradas']])

TeachersCrossMerged = pd.DataFrame(1, index=professores, columns=colunas)

for professor in professores:
  for topico in topicos:
    if topico in TrainDataframe.columns and professor in TrainDataframe.index:
      tcc_value = TrainDataframe.at[professor, topico]
      TeachersCrossMerged.at[professor, (topico, 'Participação')] = tcc_value
    if topico in TopicosHorasDataframe.columns and professor in TopicosHorasDataframe.index:
      hours_value = TopicosHorasDataframe.at[professor, topico]
      TeachersCrossMerged.at[professor, (topico, 'Horas Ministradas')] = hours_value

TeachersCrossMerged

processamento_de_linguagem_natural  \
                                                               Participação   
FRANCISCA RAQUEL DE VASCONCELOS SILVEIRA                           0.160432   
RICARDO LENZ CESAR                                                 0.013537   
KILBERT AMORIM MACIEL                                              0.001000   
SILAS SANTIAGO LOPES PEREIRA                                       0.001000   
HENRIQUE VIANA OLIVEIRA                                            0.229794   
DIEGO ROCHA LIMA                                                   0.067309   
CARINA TEIXEIRA DE OLIVEIRA                                        0.079722   
ANTONIO MAURO BARBOSA DE OLIVEIRA                                  0.005757   
MARIO WEDNEY DE LIMA MOREIRA                                       0.183902   
JOSE WILLIAM ALVES DA SILVA                                        0.001000   
ALEXANDRO LIMA DAMASCENO                                           0.005707   
ANDRESSA BEZERRA FERREIRA                                          0.001000   
RAIMUNDO VALTER COSTA FILHO                                        0.093063   
FELIPE BASTOS NUNES                                                0.005707   
THIAGO FELIPPE DE LIMA BANDEIRA                                    0.011163   
ERICA DE LIMA GALLINDO                                             0.079911   
REINALDO BEZERRA BRAGA                                             0.076995   

                                                            \
                                         Horas Ministradas   
FRANCISCA RAQUEL DE VASCONCELOS SILVEIRA          0.190967   
RICARDO LENZ CESAR                                0.000792   
KILBERT AMORIM MACIEL                             0.000792   
SILAS SANTIAGO LOPES PEREIRA                      0.444532   
HENRIQUE VIANA OLIVEIRA                           0.000792   
DIEGO ROCHA LIMA                                  0.000792   
CARINA TEIXEIRA DE OLIVEIRA                       0.064184   
ANTONIO MAURO BARBOSA DE OLIVEIRA                 0.000792   
MARIO WEDNEY DE LIMA MOREIRA                      0.000792   
JOSE WILLIAM ALVES DA SILVA                       0.000792   
ALEXANDRO LIMA DAMASCENO                          0.000792   
ANDRESSA BEZERRA FERREIRA                         0.000792   
RAIMUNDO VALTER COSTA FILHO                       0.254358   
FELIPE BASTOS NUNES                               0.000792   
THIAGO FELIPPE DE LIMA BANDEIRA                   0.000792   
ERICA DE LIMA GALLINDO                            0.000792   
REINALDO BEZERRA BRAGA                            0.000792   

                                         arquitetura_de_computadores  \
                                                        Participação   
FRANCISCA RAQUEL DE VASCONCELOS SILVEIRA                    0.016516   
RICARDO LENZ CESAR                                          0.037580   
KILBERT AMORIM MACIEL                                       0.001000   
SILAS SANTIAGO LOPES PEREIRA                                0.082497   
HENRIQUE VIANA OLIVEIRA                                     0.001000   
DIEGO ROCHA LIMA                                            0.194761   
CARINA TEIXEIRA DE OLIVEIRA                                 0.119019   
ANTONIO MAURO BARBOSA DE OLIVEIRA                           0.026263   
MARIO WEDNEY DE LIMA MOREIRA                                0.098297   
JOSE WILLIAM ALVES DA SILVA                                 0.018452   
ALEXANDRO LIMA DAMASCENO                                    0.081209   
ANDRESSA BEZERRA FERREIRA                                   0.001000   
RAIMUNDO VALTER COSTA FILHO                                 0.001000   
FELIPE BASTOS NUNES                                         0.113009   
THIAGO FELIPPE DE LIMA BANDEIRA                             0.008205   
ERICA DE LIMA GALLINDO                                      0.080649   
REINALDO BEZERRA BRAGA                                      0.13

In [ ]:
#GERAR A RECOMENDAÇÃO
def gerar_recomendacao1(assunto1, assunto2, assunto3):
  recomendacao = {}
  for index, row in TeachersCrossMerged.iterrows():
    P_Assunto1 = TeachersCrossMerged.loc[index, (assunto1, 'Participação')]
    H_Assunto1 = TeachersCrossMerged.loc[index, (assunto1, 'Horas Ministradas')]

    P_Assunto2 = TeachersCrossMerged.loc[index, (assunto2, 'Participação')]
    H_Assunto2 = TeachersCrossMerged.loc[index, (assunto2, 'Horas Ministradas')]

    P_Assunto3 = TeachersCrossMerged.loc[index, (assunto3, 'Participação')]
    H_Assunto3 = TeachersCrossMerged.loc[index, (assunto3, 'Horas Ministradas')]


    P_total = df_porcentagens_orientadores.at[index, 'Porcentagem de Aparição']

    P = P_total * P_Assunto1 * P_Assunto2 * P_Assunto3 * H_Assunto1 * H_Assunto2 * H_Assunto3
    recomendacao[index] = P

  return recomendacao


In [ ]:
def algoritmo_recomendacao1(assunto1, assunto2, assunto3, qtd):
  previsao = gerar_recomendacao1(assunto1, assunto2, assunto3)
  series = pd.Series(previsao)
  melhores_professores = series.nlargest(qtd)
  return melhores_professores


FELIPE BASTOS NUNES                         6.721061e-07
SILAS SANTIAGO LOPES PEREIRA                5.144167e-08
DIEGO ROCHA LIMA                            3.137120e-09
CARINA TEIXEIRA DE OLIVEIRA                 2.494909e-10
FRANCISCA RAQUEL DE VASCONCELOS SILVEIRA    9.663159e-12
dtype: float64

In [ ]:
TestDataset = TestTCCsDataset.drop('Orientador', axis=1)
TestDataset = TestDataset.drop('Coorientador', axis=1)
TestTCCsDataset

In [ ]:
def top3_cols(row):
    top3 = row.nlargest(3)
    return top3.index

mainSubject = TestDataset.apply(top3_cols, axis=1)
mainSubject

48    Index(['projeto_de_interface_com_usuario', 'in...
67    Index(['desenvolvimento_web', 'linguagens_de_p...
71    Index(['protocolos_de_comunicacao', 'comunicac...
15    Index(['desenvolvimento_de_aplicativos_moveis'...
39    Index(['comunicacao_sem_fio', 'arquitetura_de_...
57    Index(['sistemas_especialistas', 'desenvolvime...
31    Index(['desenvolvimento_de_aplicativos_moveis'...
29    Index(['complexidade_computacional', 'protocol...
24    Index(['complexidade_computacional', 'padroes_...
33    Index(['compiladores_e_interpretadores', 'para...
41    Index(['aprendizado_de_maquina', 'processament...
75    Index(['desenvolvimento_web', 'modelo_relacion...
5     Index(['padroes_e_boas_praticas_de_codificacao...
7     Index(['teoria_dos_grafos', 'algoritmos_de_gra...
26    Index(['redes_neurais_artificiais', 'linguagen...
58    Index(['linguagens_de_programacao_de_alto_nive...
13    Index(['desenvolvimento_web', 'bibliotecas_gra...
37    Index(['protocolos_de_comunicacao', 'siste

Teste de acurácia com o dataset de teste

In [ ]:
def teste_acuracia(n_professores):
  contador = 0
  for index, row in TestTCCsDataset.iterrows():
    assunto = mainSubject[index]
    assunto_1 = assunto[0]
    assunto_2 = assunto[1]
    assunto_3 = assunto[2]
    orientador = TestTCCsDataset.loc[index,'Orientador']
    coorientador = TestTCCsDataset.loc[index,'Coorientador']

    recomendacao = algoritmo_recomendacao1(assunto_1, assunto_2, assunto_3, n_professores)
    if orientador in recomendacao.index:
      contador += 1
    else:
      if coorientador in recomendacao.index:
        contador += 1

  tamanho = TestTCCsDataset.shape
  print("Levando em conta os ", n_professores," melhores: Dentre os", tamanho[0],", o algorítimo acertou", contador)

  acuracia = contador/tamanho[0]
  print("A acurácia foi de:", acuracia)

In [ ]:
teste_acuracia(1)
teste_acuracia(2)
teste_acuracia(3)
teste_acuracia(4)
teste_acuracia(5)

Levando em conta os  1  melhores: Dentre os 18 , o algorítimo acertou 4
A acurácia foi de: 0.2222222222222222
Levando em conta os  2  melhores: Dentre os 18 , o algorítimo acertou 7
A acurácia foi de: 0.3888888888888889
Levando em conta os  3  melhores: Dentre os 18 , o algorítimo acertou 10
A acurácia foi de: 0.5555555555555556
Levando em conta os  4  melhores: Dentre os 18 , o algorítimo acertou 14
A acurácia foi de: 0.7777777777777778
Levando em conta os  5  melhores: Dentre os 18 , o algorítimo acertou 14
A acurácia foi de: 0.7777777777777778


#CrossValidation

SEPARAÇÃO DO DATAFRAME EM 4 FOLDS

In [ ]:
TCCsDataset

,Orientador,Coorientador,complexidade_computacional,teoria_da_recursao,teoria_dos_grafos,computacao_quantica,maquina_de_turing,linguagens_recursivas_e_recursivamente_enumeraveis,teoria_da_lambda_calculabilidade,teoria_da_redutibilidade,...,seguranca_em_redes_de_computadores,mineracao_de_dados,probabilidade_e_estatistica,aprendizado_de_maquina,aprendizado_profundo,redes_neurais_artificiais,sistemas_especialistas,processamento_de_linguagem_natural,visao_computacional,algoritmos_geneticos
0,FELIPE BASTOS NUNES,MARIO WEDNEY DE LIMA MOREIRA,0.010309,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,...,0.000000,0.000000,0.020619,0.000000,0.000000,0.020619,0.000000,0.000000,0.000000,0.000000
1,RICARDO LENZ CESAR,-,0.000000,0.0,0.009346,0.0,0.000000,0.0,0.0,0.000000,...,0.000000,0.000000,0.018692,0.018692,0.009346,0.000000,0.009346,0.028037,0.000000,0.000000
2,HENRIQUE VIANA OLIVEIRA,-,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,...,0.000000,0.016393,0.000000,0.016393,0.000000,0.008197,0.000000,0.000000,0.000000,0.000000
3,MARIO WEDNEY DE LIMA MOREIRA,-,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,...,0.000000,0.000000,0.000000,0.062500,0.062500,0.187500,0.000000,0.000000,0.062500,0.000000
4,THIAGO FELIPPE DE LIMA BANDEIRA,DIEGO ROCHA LIMA,0.144886,0.0,0.008523,0.0,0.000000,0.0,0.0,0.000000,...,0.000000,0.005682,0.002841,0.190341,0.079545,0.008523,0.022727,0.022727,0.005682,0.022727
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,FRANCISCA RAQUEL DE VASCONCELOS SILVEIRA,-,0.000000,0.0,0.020642,0.0,0.000000,0.0,0.0,0.000000,...,0.000000,0.000000,0.002294,0.034404,0.009174,0.009174,0.006881,0.098624,0.000000,0.002294
78,FRANCISCA RAQUEL DE VASCONCELOS SILVEIRA,-,0.133333,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.011111,0.055556,0.000000,0.000000,0.055556
79,FRANCISCA RAQUEL DE VASCONCELOS SILVEIRA,-,0.000000,0.0,0.010657,0.0,0.000000,0.0,0.0,0.039076,...,0.000000,0.000000,0.003552,0.000000,0.000000,0.005329,0.001776,0.000000,0.000000,0.000000
80,REINALDO BEZERRA BRAGA,-,0.000000,0.0,0.000000,0.0,0.011494,0.0,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.022989,0.022989,0.000000,0.011494,0.000000


In [ ]:
#PRIMEIRO FOLD
contagem_orientadores = {}
contagem_orientadores = TCCsDataset['Orientador'].value_counts()

FoldTCCsDataset1 = {}

orientador_proportions = contagem_orientadores / contagem_orientadores.sum()
artigos_por_orientador = (orientador_proportions * 41).round().astype(int)
FoldTCCsDataset1 = pd.DataFrame()

for orientador, num_artigos in artigos_por_orientador.items():
    orientador_data = TCCsDataset[TCCsDataset['Orientador'] == orientador].sample(n=num_artigos, random_state=13)
    FoldTCCsDataset1 = pd.concat([FoldTCCsDataset1, orientador_data])

#PREPARAÇÃO DO RESTANTE DO DATAFRAME DE TREINO
indices_itens_teste = TCCsDataset.index.isin(FoldTCCsDataset1.index)
SplitTCCsDataset1 = TCCsDataset[~indices_itens_teste]

FoldTCCsDataset1.shape

(38, 88)

In [ ]:
#SEGUNDO FOLD
contagem_orientadores = {}
contagem_orientadores = SplitTCCsDataset1['Orientador'].value_counts()

FoldTCCsDataset2 = {}

orientador_proportions = contagem_orientadores / contagem_orientadores.sum()
artigos_por_orientador = (orientador_proportions * 22).round().astype(int)
FoldTCCsDataset2 = pd.DataFrame()

for orientador, num_artigos in artigos_por_orientador.items():
    orientador_data = SplitTCCsDataset1[SplitTCCsDataset1['Orientador'] == orientador].sample(n=num_artigos, random_state=13)
    FoldTCCsDataset2 = pd.concat([FoldTCCsDataset2, orientador_data])

#PREPARAÇÃO DO RESTANTE DO DATAFRAME DE TREINO
indices_itens_teste = SplitTCCsDataset1.index.isin(FoldTCCsDataset2.index)
SplitTCCsDataset2 = SplitTCCsDataset1[~indices_itens_teste]

FoldTCCsDataset2.shape

(21, 88)

In [ ]:
#TERCEIRO FOLD
contagem_orientadores = {}
contagem_orientadores = FoldTCCsDataset1['Orientador'].value_counts()

FoldTCCsDataset3 = {}

orientador_proportions = contagem_orientadores / contagem_orientadores.sum()
artigos_por_orientador = (orientador_proportions * 21).round().astype(int)
FoldTCCsDataset3 = pd.DataFrame()

for orientador, num_artigos in artigos_por_orientador.items():
    orientador_data = FoldTCCsDataset1[FoldTCCsDataset1['Orientador'] == orientador].sample(n=num_artigos, random_state=13)
    FoldTCCsDataset3 = pd.concat([FoldTCCsDataset3, orientador_data])

#PREPARAÇÃO DO RESTANTE DO DATAFRAME DE TREINO
indices_itens_teste = FoldTCCsDataset1.index.isin(FoldTCCsDataset3.index)
SplitTCCsDataset3 = FoldTCCsDataset1[~indices_itens_teste]

FoldTCCsDataset3.shape

(20, 88)

In [ ]:
SplitTCCsDataset2

In [ ]:
#Organizar folds
print(SplitTCCsDataset3.shape, FoldTCCsDataset2.shape, FoldTCCsDataset3.shape, SplitTCCsDataset2.shape)

(20, 88) (21, 88) (20, 88) (21, 88)


In [ ]:
FoldTCCsDataset1 = SplitTCCsDataset3
FoldTCCsDataset2 = FoldTCCsDataset2
FoldTCCsDataset3 = FoldTCCsDataset3
FoldTCCsDataset4 = SplitTCCsDataset2

In [ ]:
print(FoldTCCsDataset1.shape, FoldTCCsDataset2.shape, FoldTCCsDataset3.shape, FoldTCCsDataset4.shape)

(20, 88) (21, 88) (20, 88) (21, 88)


In [ ]:
def algoritmo_naive_bayes(trainDataframe, testDataframe, peso_hora, peso_orientacao):
  PontuacaoTCC = trainDataframe
  dataset_columns = PontuacaoTCC.columns[2:]
  dataset_columns

  total_professores = {}

  for index, row in PontuacaoTCC.iterrows():
    orientador = row['Orientador']
    coorientador = row['Coorientador']


    if orientador not in total_professores:
      total_professores[orientador] = {}
      total_professores[orientador] = row[dataset_columns]
    elif orientador in total_professores:
      total_professores[orientador] += row[dataset_columns]

    if coorientador not in total_professores:
      total_professores[coorientador] = {}
      total_professores[coorientador] = row[dataset_columns]
    elif coorientador in total_professores:
      total_professores[coorientador] += row[dataset_columns]


  total_orientadores_df = pd.DataFrame(total_professores)
  total_orientadores_df = total_orientadores_df.drop(columns = '-', axis=1)

  PontuacaoTCC = total_orientadores_df

  def calcular_porcentagem(linha):
      soma = linha.sum()
      if soma == 0:
          return linha
      return (linha / soma)

  dataset_columns = PontuacaoTCC.apply(calcular_porcentagem, axis=1)

  Database = dataset_columns.transpose()
  Database = Database + 0.001

  #DATASET HORAS
  professores = list(set(Database.index))
  topicos = list(set(Database.columns))

  colunas = pd.MultiIndex.from_product([topicos, ['Participação', 'Horas Ministradas']])

  TeachersCrossMerged = pd.DataFrame(1, index=professores, columns=colunas)

  for professor in professores:
    for topico in topicos:
      if topico in Database.columns and professor in Database.index:
        tcc_value = Database.at[professor, topico]
        TeachersCrossMerged.at[professor, (topico, 'Participação')] = tcc_value
      if topico in TopicosHorasDataframe.columns and professor in TopicosHorasDataframe.index:
        hours_value = TopicosHorasDataframe.at[professor, topico]
        TeachersCrossMerged.at[professor, (topico, 'Horas Ministradas')] = hours_value

  TeachersCrossMerged


  #Algoritmo
  def algoritmo(assunto1, assunto2, assunto3):
    recomendacao = {}
    for index, row in TeachersCrossMerged.iterrows():
      P_Assunto1 = TeachersCrossMerged.loc[index, (assunto1, 'Participação')] * peso_orientacao
      H_Assunto1 = TeachersCrossMerged.loc[index, (assunto1, 'Horas Ministradas')] * peso_hora

      P_Assunto2 = TeachersCrossMerged.loc[index, (assunto2, 'Participação')] * peso_orientacao
      H_Assunto2 = TeachersCrossMerged.loc[index, (assunto2, 'Horas Ministradas')] * peso_hora

      P_Assunto3 = TeachersCrossMerged.loc[index, (assunto3, 'Participação')] * peso_orientacao
      H_Assunto3 = TeachersCrossMerged.loc[index, (assunto3, 'Horas Ministradas')] * peso_hora

      P_total = df_porcentagens_orientadores.at[index, 'Porcentagem de Aparição']

      if H_Assunto1 == 0:
        P = P_total * P_Assunto1 * P_Assunto2 * P_Assunto3
        recomendacao[index] = P
      elif P_Assunto1 == 0:
        P = P_total * H_Assunto1 * H_Assunto2 * H_Assunto3
        recomendacao[index] = P
      else:
        P = P_total * P_Assunto1 * P_Assunto2 * P_Assunto3 * H_Assunto1 * H_Assunto2 * H_Assunto3
        recomendacao[index] = P

    return recomendacao

  #CHAMA A FUNÇÃO E TRAZ OS 5 PROFESSORES COM MAIOR PROBABILIDADE
  def funcao_calc(assunto1, assunto2, assunto3, qtd):
    previsao = algoritmo(assunto1, assunto2, assunto3)
    series = pd.Series(previsao)
    melhores_professores = series.nlargest(qtd)
    return melhores_professores

  TestDataset = testDataframe.drop('Orientador', axis=1)
  TestDataset = TestDataset.drop('Coorientador', axis=1)

  #TRAZ OS 3 PRINCIPAIS TEMAS DO TCC
  def top3_cols(row):
    top3 = row.nlargest(3)
    return top3.index

  mainSubject = TestDataset.apply(top3_cols, axis=1)

  #CHAMA A FUNÇÃO PARA CALCULAR A ACURÁCIA
  def teste_validacao(n_professores):
    contador = 0
    for index, row in testDataframe.iterrows():
      assunto = mainSubject[index]
      assunto_1 = assunto[0]
      assunto_2 = assunto[1]
      assunto_3 = assunto[2]
      orientador = testDataframe.loc[index,'Orientador']
      coorientador = testDataframe.loc[index,'Coorientador']

      recomendacao = funcao_calc(assunto_1, assunto_2, assunto_3, n_professores)
      if orientador in recomendacao.index:
        contador += 1
      elif coorientador in recomendacao.index:
        contador += 1
      # return (recomendacao,orientador)

    tamanho = testDataframe.shape
    print("Levando em conta os ", n_professores," melhores: Dentre os", tamanho[0],", o algorítimo acertou", contador)
    acuracia = contador/tamanho[0]
    print("A acurácia foi de:", acuracia)

  teste_validacao(1)
  teste_validacao(2)
  teste_validacao(3)
  teste_validacao(4)
  teste_validacao(5)


In [ ]:
def crossValidation(peso_hours, peso_orientation):
  print('Utilizando preso hora:', peso_hours, 'peso orientacao:', peso_orientation)

  round1TrainDataframe = {}
  round1TrainDataframe = pd.concat([FoldTCCsDataset2, FoldTCCsDataset3, FoldTCCsDataset4])
  round1TestDataframe = {}
  round1TestDataframe = FoldTCCsDataset1

  round2TrainDataframe = {}
  round2TrainDataframe = pd.concat([FoldTCCsDataset1, FoldTCCsDataset3, FoldTCCsDataset4])
  round2TestDataframe = {}
  round2TestDataframe = FoldTCCsDataset2

  round3TrainDataframe = {}
  round3TrainDataframe = pd.concat([FoldTCCsDataset1, FoldTCCsDataset2, FoldTCCsDataset4])
  round3TestDataframe = {}
  round3TestDataframe = FoldTCCsDataset3

  round4TrainDataframe = {}
  round4TrainDataframe = pd.concat([FoldTCCsDataset1, FoldTCCsDataset2, FoldTCCsDataset3])
  round4TestDataframe = {}
  round4TestDataframe = FoldTCCsDataset4

  print("Primeira Rodada:")
  algoritmo_naive_bayes(round1TrainDataframe, round1TestDataframe, peso_hours, peso_orientation)

  print("Segunda Rodada:")
  algoritmo_naive_bayes(round2TrainDataframe, round2TestDataframe, peso_hours, peso_orientation)

  print("Terceira Rodada:")
  algoritmo_naive_bayes(round3TrainDataframe, round3TestDataframe, peso_hours, peso_orientation)

  print("Quarta Rodada:")
  algoritmo_naive_bayes(round4TrainDataframe, round4TestDataframe, peso_hours, peso_orientation)


In [ ]:
crossValidation(1, 0)

Utilizando preso hora: 1 peso orientacao: 0
Primeira Rodada:
Levando em conta os  1  melhores: Dentre os 20 , o algorítimo acertou 4
A acurácia foi de: 0.2
Levando em conta os  2  melhores: Dentre os 20 , o algorítimo acertou 8
A acurácia foi de: 0.4
Levando em conta os  3  melhores: Dentre os 20 , o algorítimo acertou 8
A acurácia foi de: 0.4
Levando em conta os  4  melhores: Dentre os 20 , o algorítimo acertou 12
A acurácia foi de: 0.6
Levando em conta os  5  melhores: Dentre os 20 , o algorítimo acertou 14
A acurácia foi de: 0.7
Segunda Rodada:
Levando em conta os  1  melhores: Dentre os 21 , o algorítimo acertou 4
A acurácia foi de: 0.19047619047619047
Levando em conta os  2  melhores: Dentre os 21 , o algorítimo acertou 9
A acurácia foi de: 0.42857142857142855
Levando em conta os  3  melhores: Dentre os 21 , o algorítimo acertou 11
A acurácia foi de: 0.5238095238095238
Levando em conta os  4  melhores: Dentre os 21 , o algorítimo acertou 13
A acurácia foi de: 0.6190476190476191
Le

In [ ]:
crossValidation(0.75, 0.25)

Utilizando preso hora: 0.75 peso orientacao: 0.25
Primeira Rodada:
Levando em conta os  1  melhores: Dentre os 20 , o algorítimo acertou 4
A acurácia foi de: 0.2
Levando em conta os  2  melhores: Dentre os 20 , o algorítimo acertou 10
A acurácia foi de: 0.5
Levando em conta os  3  melhores: Dentre os 20 , o algorítimo acertou 11
A acurácia foi de: 0.55
Levando em conta os  4  melhores: Dentre os 20 , o algorítimo acertou 13
A acurácia foi de: 0.65
Levando em conta os  5  melhores: Dentre os 20 , o algorítimo acertou 14
A acurácia foi de: 0.7
Segunda Rodada:
Levando em conta os  1  melhores: Dentre os 21 , o algorítimo acertou 6
A acurácia foi de: 0.2857142857142857
Levando em conta os  2  melhores: Dentre os 21 , o algorítimo acertou 10
A acurácia foi de: 0.47619047619047616
Levando em conta os  3  melhores: Dentre os 21 , o algorítimo acertou 13
A acurácia foi de: 0.6190476190476191
Levando em conta os  4  melhores: Dentre os 21 , o algorítimo acertou 17
A acurácia foi de: 0.809523809

In [ ]:
crossValidation(0.5, 0.5)

Utilizando preso hora: 0.5 peso orientacao: 0.5
Primeira Rodada:
Levando em conta os  1  melhores: Dentre os 20 , o algorítimo acertou 4
A acurácia foi de: 0.2
Levando em conta os  2  melhores: Dentre os 20 , o algorítimo acertou 10
A acurácia foi de: 0.5
Levando em conta os  3  melhores: Dentre os 20 , o algorítimo acertou 11
A acurácia foi de: 0.55
Levando em conta os  4  melhores: Dentre os 20 , o algorítimo acertou 13
A acurácia foi de: 0.65
Levando em conta os  5  melhores: Dentre os 20 , o algorítimo acertou 14
A acurácia foi de: 0.7
Segunda Rodada:
Levando em conta os  1  melhores: Dentre os 21 , o algorítimo acertou 6
A acurácia foi de: 0.2857142857142857
Levando em conta os  2  melhores: Dentre os 21 , o algorítimo acertou 10
A acurácia foi de: 0.47619047619047616
Levando em conta os  3  melhores: Dentre os 21 , o algorítimo acertou 13
A acurácia foi de: 0.6190476190476191
Levando em conta os  4  melhores: Dentre os 21 , o algorítimo acertou 17
A acurácia foi de: 0.80952380952

In [ ]:
crossValidation(0.25, 0.75)

Utilizando preso hora: 0.25 peso orientacao: 0.75
Primeira Rodada:
Levando em conta os  1  melhores: Dentre os 20 , o algorítimo acertou 4
A acurácia foi de: 0.2
Levando em conta os  2  melhores: Dentre os 20 , o algorítimo acertou 10
A acurácia foi de: 0.5
Levando em conta os  3  melhores: Dentre os 20 , o algorítimo acertou 11
A acurácia foi de: 0.55
Levando em conta os  4  melhores: Dentre os 20 , o algorítimo acertou 13
A acurácia foi de: 0.65
Levando em conta os  5  melhores: Dentre os 20 , o algorítimo acertou 14
A acurácia foi de: 0.7
Segunda Rodada:
Levando em conta os  1  melhores: Dentre os 21 , o algorítimo acertou 6
A acurácia foi de: 0.2857142857142857
Levando em conta os  2  melhores: Dentre os 21 , o algorítimo acertou 10
A acurácia foi de: 0.47619047619047616
Levando em conta os  3  melhores: Dentre os 21 , o algorítimo acertou 13
A acurácia foi de: 0.6190476190476191
Levando em conta os  4  melhores: Dentre os 21 , o algorítimo acertou 17
A acurácia foi de: 0.809523809

In [ ]:
crossValidation(0, 1)

Utilizando preso hora: 0 peso orientacao: 1
Primeira Rodada:
Levando em conta os  1  melhores: Dentre os 20 , o algorítimo acertou 6
A acurácia foi de: 0.3
Levando em conta os  2  melhores: Dentre os 20 , o algorítimo acertou 11
A acurácia foi de: 0.55
Levando em conta os  3  melhores: Dentre os 20 , o algorítimo acertou 14
A acurácia foi de: 0.7
Levando em conta os  4  melhores: Dentre os 20 , o algorítimo acertou 14
A acurácia foi de: 0.7
Levando em conta os  5  melhores: Dentre os 20 , o algorítimo acertou 16
A acurácia foi de: 0.8
Segunda Rodada:
Levando em conta os  1  melhores: Dentre os 21 , o algorítimo acertou 6
A acurácia foi de: 0.2857142857142857
Levando em conta os  2  melhores: Dentre os 21 , o algorítimo acertou 11
A acurácia foi de: 0.5238095238095238
Levando em conta os  3  melhores: Dentre os 21 , o algorítimo acertou 14
A acurácia foi de: 0.6666666666666666
Levando em conta os  4  melhores: Dentre os 21 , o algorítimo acertou 16
A acurácia foi de: 0.7619047619047619
